In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.plotting.register_matplotlib_converters()
%matplotlib inline

import duckdb


%load_ext autoreload
%autoreload 2
from help_tool import help_tool

#pd.set_option('future.no_silent_downcasting', True)

# Setting graph parameters
sns.set_theme(style='white')

pd.set_option('display.max_colwidth', None)

In [2]:
train = help_tool.csv_download(
    r'Archive\train.csv')

help_tool.first_look(train)

Dataset has 159571 observations and 8 features
Columns with all empty values []
Dataset has 0 duplicates


,dtypes,"Null values, Count","Null values, %"
id,object,0,0.0
comment_text,object,0,0.0
toxic,int64,0,0.0
severe_toxic,int64,0,0.0
obscene,int64,0,0.0
threat,int64,0,0.0
insult,int64,0,0.0
identity_hate,int64,0,0.0


In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


# Cleaning Data

Title is removed. not

In [4]:
train[train['id'] == '83e37ee1eb4be93e']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
49323,83e37ee1eb4be93e,"""== Blocked as a sockpuppet ==\n""",1,0,0,0,0,0


In [5]:
train[train['comment_text'].str.contains(r'==.*==\n', regex=True)]['comment_text'].head(1)

29    "== A barnstar for you! ==\n\n  The Real Life Barnstar lets us be the stars\n   "
Name: comment_text, dtype: object

In [6]:
train['comment_text'] = train['comment_text'].replace({r'\n': '.', r'==': ''}, regex=True)

# Dublicated values

In [7]:
train[train.duplicated(['comment_text'], keep=False)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
45381,7959f269d6d08626,"Blocked.I am blocked by what I consider to be immature aggression by abusing his perogatives. I would like for him to remove the block and, if he feels it is absoloutely necessary to have me blocked to have it done by a neutral third party.",0,0,0,0,0,0
130949,bca271fe50a87e06,"Blocked.I am blocked by what I consider to be immature aggression by abusing his perogatives. I would like for him to remove the block and, if he feels it is absoloutely necessary to have me blocked to have it done by a neutral third party.",0,0,0,0,0,0


In [8]:
train = train[~train.duplicated(['comment_text'])]

In [9]:
train.shape

(159570, 8)

## Time stamps

In [49]:
combined_pattern = (
    r'\b\d{1,2}, \d{1,2} [A-Za-z]{3,10} \d{4}\b|'  # "31, 19 July 2006"
    r'\b\d{1,2} \d{1,2} [A-Za-z]{3,10} \d{4}\b|'  # "31 19 July 2006"
    r'\b\d{1,2} [A-Za-z]{3,10}, \d{4}\b|'  # "31 July, 2006"
    r'\b\d{1,2} [A-Za-z]{3,10} \d{4}\b|'  # "31 July 2006"
    r'\b\d{1,2}, [A-Za-z]{3,10} \d{4}, \d{1,2}:\d{2}\b|'  # "31, July 2006, 18:47"
    r'\b\d{1,2} [A-Za-z]{3,10} \d{4}, \d{1,2}:\d{2}\b|'  # "31 July 2006, 18:47"
    r'\b\d{1,2} [A-Za-z]{3,10} \d{4} \d{1,2}:\d{2}\b|'  # "31 July 2006 18:47"
    r'\b\d{1,2}:\d{2}, [A-Za-z]{3,10} \d{1,2}, \d{4}\b|'  # "18:47, July 31, 2006"
    r'\b\d{1,2}:\d{2}, \d{1,2} [A-Za-z]{3,10} \d{4}\b|'  # "18:47, 31 July 2006"
    r'\b\d{1,2}:\d{2} \d{1,2} [A-Za-z]{3,10} \d{4}\b|'  # "18:47 31 July 2006"
    r'\b\d{1,2} [A-Za-z]{3,10} \d{1,2}:\d{2}\b|' # "26 July 17:03 UTC"
    r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b|' # API adress
    r'\(UTC\)|\(talk\)|\(\)|'
    r'(talk.*)|'
    r'Preceding unsigned comment added by'
)


# Remove the date pattern from the 'comment_text' column
train['comment_text_cleaned'] = train['comment_text'].str.replace(combined_pattern, '', regex=True).str.strip()

train['comment_text_cleaned'].iloc[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks."

In [50]:
train = train[train['comment_text_cleaned'] != '']

# English leters

In [51]:
def remove_non_ascii(text):
    return ''.join([char for char in text if ord(char) <= 127])

train['cleaned_text'] = train['comment_text_cleaned'].apply(remove_non_ascii)

In [52]:
train['cleaned_text'] = train['cleaned_text'].str.lstrip(' ').str.lstrip('"').str.lstrip('.').str.lstrip('(')
train['cleaned_text'] = train['cleaned_text'].replace({'USER': '', 'User': '', 'REDIRECT User': ''})
train['cleaned_text'] = train['cleaned_text'].str.lstrip(' ').str.lstrip('"').str.lstrip('.').str.lstrip('(')
train = train[train['cleaned_text'] != '']

In [53]:
train[train['cleaned_text'].duplicated()][['cleaned_text']].drop_duplicates().head(50)

,cleaned_text
1233,"Please stop. If you continue to vandalize Wikipedia, you will be blocked from editing."
1736,"Welcome!..Hello, , and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages that you might find helpful:.The five pillars of Wikipedia.How to edit a page.Help pages.Tutorial.How to write a great article.Manual of Style.I hope you enjoy editing here and being a Wikipedian! Please sign your name on"
3353,"Welcome..Hello, and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are some pages you might like to see:.. The five pillars of Wikipedia. How to edit a page. Help pages. Tutorial. How to write a great article. Manual of Style..You are welcome to continue editing articles without logging in, but you may wish to [ create an account]. Doing so is free, requires no personal information, and provides several benefits. If you edit without a username, your IP address is used to identify you instead...In any case, I hope you enjoy editing here and being a Wikipedian! Please sign your comments on"
4927,"Welcome!..Welcome to Wikipedia, ! My name is Ryan, aka . I noticed that you were new and haven't received any messages yet. I just wanted to see how you were doing. Wikipedia can be a little intimidating at first, since it uses different formatting than other sites that use HTML and CSS. In the long run, though, you'll find that the WikiSyntax is a lot easier and faster than those other ways. Here are a few links to get you started:..How to edit a page.Editing, policy, conduct, and structure tutorial.Picture tutorial.How to write a great article..There are a lot of policies and guides to read, but I highly recommend reading over those first. If you have any questions, feel free to leave me a message on my"
5926,"Welcome..Hello, and welcome to Wikipedia! Thank you for your contributions. I hope you like the place and decide to stay. Here are a few good links for newcomers:.. The five pillars of Wikipedia. How to edit a page. Help pages. Tutorial. How to write a great article. Manual of Style..Here are some other hints and tips:.. I would recommend that you get a username. You don't have to log in to read or edit articles on Wikipedia, but creating an account is quick, free and non-intrusive, requires no personal information, and there are many benefits of having a username. (If you edit without a username, your IP address is used to identify you instead.). When using"
6448,Editing pages ..To anyone who reads Dmacks'
7491,You
7572,"It might not have been your intention, but your recent edit removed content from . Please be careful not to remove content from Wikipedia without a valid reason, which you should specify in the edit summary or on the article's"
8634,"{| style=""""background-color:#F5FFFA; padding:0;"""" cellpadding=""""0"""".|style=""""border:1px solid #084080; background-color:#F5FFFA; vertical-align:top; color:#000000;""""|. Hello, ! Welcome to Wikipedia! Thank you for your contributions to this free encyclopedia. If you decide that you need help, check out Getting Help below, ask me on , or place on your"
9258,"Please do not vandalize pages, as you did with this edit to User"


In [18]:
# train = train[~train['cleaned_text'].duplicated()]

In [20]:
train = train[~train.duplicated(['comment_text_cleaned'])]

In [21]:
train.shape

(159242, 9)

In [22]:
train = train[train['comment_text_cleaned'] != '']

In [23]:
train.shape

(159241, 9)

# Label Distirbution

In [24]:
train[
    (train['toxic'] == 0)
    &
    (train['severe_toxic'] == 0)
    &
    (train['obscene'] == 0)
    &
    (train['threat'] == 0)
    &
    (train['insult'] == 0)
    &
    (train['identity_hate'] == 0)
      ].shape[0] / train.shape[0] * 100

89.8229727268731

In [25]:
train.drop(columns=['id', 'comment_text']).sum() / train.shape[0] * 100